In [1]:
# %pip install sklearn

In [2]:
import numpy as np
import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.metrics import r2_score
# from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [3]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_rC4_1_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_1_75.csv')
df_rC4_5_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_5_75.csv')




                  Date  425_pct_Al  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-16 16:00:00    6.307630    2.508420    21.96340   
1  2012-05-16 18:00:00    6.308335    2.511135    21.96865   
2  2012-05-16 19:00:00    6.309390    2.515210    21.97655   
3  2012-05-16 20:00:00    6.310090    2.517925    21.98185   
4  2012-05-16 21:00:00    6.310790    2.520640    21.98715   

   HydWtr_pct_Ammonia  HydWtr_Na2O    Butanol   Decanol   DI55102   DI55152  \
0            0.908765     2.712180  54.858300  0.722511  0.970496  0.925255   
1            0.907643     2.805415  54.858300  0.722511  0.970496  0.925255   
2            0.905959     2.945270  54.858300  0.722511  0.970496  0.925255   
3            0.904837     3.038505  51.190050  0.733904  0.970653  0.923773   
4            0.903715     3.131740  48.744567  0.741499  0.970307  0.924103   

   ...    LC55557    LC90366    LC90368   PI55004   PI55020  TC55552  \
0  ...  65.873400  61.011100  51.234400  1.306310 -0.607993  177.188   
1  ...  

In [8]:
print(df_rC4_1_75.info())
print(df_rC4_5_75.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47882 entries, 0 to 47881
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                47882 non-null  object 
 1   425_pct_Al          47882 non-null  float64
 2   C4_pct_Eth          47882 non-null  float64
 3   C4_pct_H2O          47882 non-null  float64
 4   HydWtr_pct_Ammonia  47882 non-null  float64
 5   HydWtr_Na2O         47882 non-null  float64
 6   Butanol             47882 non-null  float64
 7   Decanol             47882 non-null  float64
 8   DI55102             47882 non-null  float64
 9   DI55152             47882 non-null  float64
 10  FC55003             47882 non-null  float64
 11  FC55009             47882 non-null  float64
 12  FC55552             47882 non-null  float64
 13  FC55569             47882 non-null  float64
 14  FC55576             47882 non-null  float64
 15  FFC55555            47882 non-null  float64
 16  LC55

In [9]:
print(df_rC4_1_75.head())
print(df_rC4_5_75.head())

                  Date  425_pct_Al  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-16 16:00:00    6.307630    2.508420    21.96340   
1  2012-05-16 18:00:00    6.308335    2.511135    21.96865   
2  2012-05-16 19:00:00    6.309390    2.515210    21.97655   
3  2012-05-16 20:00:00    6.310090    2.517925    21.98185   
4  2012-05-16 21:00:00    6.310790    2.520640    21.98715   

   HydWtr_pct_Ammonia  HydWtr_Na2O    Butanol   Decanol   DI55102   DI55152  \
0            0.908765     2.712180  54.858300  0.722511  0.970496  0.925255   
1            0.907643     2.805415  54.858300  0.722511  0.970496  0.925255   
2            0.905959     2.945270  54.858300  0.722511  0.970496  0.925255   
3            0.904837     3.038505  51.190050  0.733904  0.970653  0.923773   
4            0.903715     3.131740  48.744567  0.741499  0.970307  0.924103   

   FC55003      FC55009       FC55552      FC55569    FC55576  FFC55555  \
0  4919.29  1528.010000  41564.100000  6918.110000  334.94800  0.751501   
1 

In [10]:
# Calculate the differences and create new columns
df_rC4_1_75['FFC55555_diff'] = df_rC4_1_75['FFC55555'] - df_rC4_1_75['FFC55555_adj']
df_rC4_1_75['Butanol_diff'] = df_rC4_1_75['Butanol'] - df_rC4_1_75['Butanol_adj']
df_rC4_1_75['Decanol_diff'] = df_rC4_1_75['Decanol'] - df_rC4_1_75['Decanol_adj']

In [11]:
# Calculate the differences and create new columns
df_rC4_5_75['FFC55555_diff'] = df_rC4_5_75['FFC55555'] - df_rC4_5_75['FFC55555_adj']
df_rC4_5_75['Butanol_diff'] = df_rC4_5_75['Butanol'] - df_rC4_5_75['Butanol_adj']
df_rC4_5_75['Decanol_diff'] = df_rC4_5_75['Decanol'] - df_rC4_5_75['Decanol_adj']

In [13]:

df_rC4_1_75['Date'] = pd.to_datetime(df_rC4_1_75['Date'])
df_rC4_1_75.set_index('Date', inplace=True)


In [14]:
# Daily averages for each _diff column
daily_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('D').mean()

# Monthly averages
monthly_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('M').mean()

# Yearly averages
yearly_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('Y').mean()

# Hourly averages
hourly_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('H').mean()


C:\Users\steve\AppData\Local\Temp\ipykernel_12908\2262192722.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('M').mean()
C:\Users\steve\AppData\Local\Temp\ipykernel_12908\2262192722.py:8: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('Y').mean()
C:\Users\steve\AppData\Local\Temp\ipykernel_12908\2262192722.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_averages = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('H').mean()


In [17]:
monthly_averages

,FFC55555_diff,Butanol_diff,Decanol_diff
Date,,,
2012-05-31,0.000076,-0.018493,-0.048897
2012-06-30,0.004050,-0.204695,-0.047093
2012-07-31,0.003943,-0.371680,-0.090372
2012-08-31,0.008015,-0.155287,-0.072003
2012-09-30,0.008860,-1.520244,-0.122041
...,...,...,...
2021-11-30,NaN,NaN,NaN
2021-12-31,NaN,NaN,NaN
2022-01-31,NaN,NaN,NaN


In [18]:
df_rC4_5_75['Date'] = pd.to_datetime(df_rC4_5_75['Date'])
df_rC4_5_75.set_index('Date', inplace=True)

In [20]:
# Daily averages for each _diff column
daily_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('D').mean()

# Monthly averages
monthly_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('M').mean()

# Yearly averages
yearly_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('Y').mean()

# Hourly averages
hourly_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('H').mean()


C:\Users\steve\AppData\Local\Temp\ipykernel_12908\2065083067.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('M').mean()
C:\Users\steve\AppData\Local\Temp\ipykernel_12908\2065083067.py:8: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('Y').mean()
C:\Users\steve\AppData\Local\Temp\ipykernel_12908\2065083067.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_averages = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff']].resample('H').mean()


In [21]:
yearly_averages

,FFC55555_diff,Butanol_diff,Decanol_diff
Date,,,
2012-12-31,0.027967,-1.430688,-0.062344
2013-12-31,0.020425,-0.575357,0.006743
2014-12-31,0.024079,-1.474933,0.009806
2015-12-31,0.022537,-1.455950,-0.047143
2016-12-31,0.020743,-1.630272,-0.040739
2017-12-31,0.024313,-1.347107,-0.087983
2018-12-31,0.002635,-0.300505,-0.043227
2019-12-31,0.005189,-0.770892,-0.052839
2020-12-31,0.015771,-2.068492,-0.055785
